In [1]:
spark.conf.set(
  "fs.azure.account.key.zbatchaistorage.blob.core.windows.net",
  "your_storage_acc_key")

In [2]:
### mounting blob to databrick only once and it will be there forever 

dbutils.fs.mount(source = "wasbs://mntdatabrick@zbatchaistorage.blob.core.windows.net/faces/",mount_point = "/mnt/root/",extra_configs = {"fs.azure.account.key.zbatchaistorage.blob.core.windows.net": "your_storage_access_key"})


--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-1277953500304291> in <module> () 
 1 ### mounting blob to databrick 
 ----> 2 dbutils . fs . mount ( source = "wasbs://mntdatabrick@zbatchaistorage.blob.core.windows.net/faces/" , mount_point = "/mnt/root/" , extra_configs = { "fs.azure.account.key.zbatchaistorage.blob.core.windows.net" : "MmkzeKphqtgv83Z3DfieWNF40bdj53FQ7WebO2zPW73Jh3YOxOgGuAkj/gi0Bo/4K+awcWtlGvpDE9tIQvKYYw==" } ) 

 /tmp/1534253315695-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 227 class ExecutionError ( Exception ) : 
 228 pass 
 --> 229 raise ExecutionError ( str ( e ) ) 
 230 return f_with_exception_handling 
 231 

 ExecutionError : An error occurred while calling o174.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/root; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/root
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:55)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:385)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:226)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/root
	at scala.Predef$.require(Predef.scala:277)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:127)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:60)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:71)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:70)
	at scala.collection.immutable.List.foreach(List.scala:389)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:70)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:272)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:252)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:42)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:58)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:58)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:38)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$2(UsageLogging.scala:313)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:188)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:183)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:180)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:13)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:221)
	a

In [3]:
%fs ls /mnt/root/female/

path,name,size
dbfs:/mnt/root/female/Angelina/,Angelina/,0
dbfs:/mnt/root/female/CateBlanchett/,CateBlanchett/,0
dbfs:/mnt/root/female/KatyPerry/,KatyPerry/,0
dbfs:/mnt/root/female/Scarlett/,Scarlett/,0


In [4]:
import os
import sys
os.listdir('/dbfs/mnt/root/')

Out[ 15 ]: 
['Godfather.jpg',
 'Sj-test.jpg',
 'angelina-jolie-test.jpg',
 'cb-test.jpg',
 'female',
 'guestwho.jpg',
 'haarcascade_frontalface_default.xml',
 'kp-test.jpg',
 'male']

In [5]:
import mmlspark
import numpy as np
from mmlspark import toNDArray
import cv2
import sys
import os
import numpy as np

size = 1
fn_haar = "/dbfs/mnt/root/haarcascade_frontalface_default.xml"
#fn_dir = 'data/male/'
 # Scale down for speed
(im_width, im_height) = (128, 128)

In [6]:
def get_images(fn_dir, images, labels, names, idx):
  # Convert to grayscale
  for (subdir,dirs, files) in os.walk(fn_dir):
    #print("subdir {}, dirs:{}, files:{}".format(subdir,dirs, files))
    for subdir in dirs:
      print("subdir",subdir)
      names[idx] = subdir
      subjectpath = os.path.join(fn_dir, subdir)
      print(subjectpath)
      # Loop through each photo in the folder
      for filename in os.listdir(subjectpath):
        # Skip non-image formates
        f_name, f_extension = os.path.splitext(filename)
        if(f_extension.lower() not in  ['.png','.jpg','.jpeg','.gif','.pgm']):
          print("Skipping "+filename+", wrong file type")
          continue
        path = subjectpath + '/' + filename
        
        img=cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mini = cv2.resize(gray, (im_width, im_height))
        # Add to training data
        images.append(mini)
        labels.append(int(idx))
      idx += 1
  return images, labels, names, idx

In [7]:
maledir='/dbfs/mnt/root/male/'
images, labels, names, idx=get_images('/dbfs/mnt/root/female/',[], [], {}, 0)
images, labels, names, idx=get_images('/dbfs/mnt/root/male/',images, labels, names, idx)

('subdir', 'Angelina')
/dbfs/mnt/root/female/Angelina
('subdir', 'CateBlanchett')
/dbfs/mnt/root/female/CateBlanchett
('subdir', 'KatyPerry')
/dbfs/mnt/root/female/KatyPerry
('subdir', 'Scarlett')
/dbfs/mnt/root/female/Scarlett
('subdir', 'AndrewNg')
/dbfs/mnt/root/male/AndrewNg
('subdir', 'JH')
/dbfs/mnt/root/male/JH

In [8]:
(images, labels) = [np.array(lis) for lis in [images, labels]]
images.shape, labels.shape

Out[ 4 ]: ((85, 128, 128), (85,))

In [9]:
names

Out[ 5 ]: 
{0: 'Angelina',
 1: 'CateBlanchett',
 2: 'KatyPerry',
 3: 'Scarlett',
 4: 'AndrewNg',
 5: 'JH'}

In [10]:
import cv2
## remember to ONLY pip install opencv-contrib-python NOT pip install opencv-python
# otherwise it will overwrite your face module
model1=cv2.face.LBPHFaceRecognizer_create()
model2=cv2.face.EigenFaceRecognizer_create()
model3=cv2.face.FisherFaceRecognizer_create()

In [11]:
model1.train(images, labels)
model2.train(images,labels)
model3.train(images,labels)

In [12]:
new_image=cv2.imread('/dbfs/mnt/root/angelina-jolie-test.jpg')
new_image.shape

Out[ 16 ]: (800, 591, 3)

In [13]:
# make prediction based on static data
haar_cascade = cv2.CascadeClassifier(fn_haar)

def detect_face(model,haar_cascade,new_img,num_faces, save_path):
  #colored=cv2.resize(new_img, (int(new_img.shape[1]*0.5), int(new_img.shape[0]*0.5)))
  # Convert to grayscalel
  grayim = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
  print("grayim shape", grayim.shape)


  # Detect faces and loop through each one
  haar_cascade = cv2.CascadeClassifier(fn_haar)
  faces = haar_cascade.detectMultiScale(grayim)
  print("face detector arr shape", faces.shape)

  face_i = faces[0]

  # Coordinates of face after scaling back by `size`
  (x, y, w, h) = [v * 1 for v in face_i]
  print("face at x:{}, y:{},w:{},h:{}".format(x,y,w,h))
  face = grayim[y:y + h, x:x + w]
  face_resize = cv2.resize(face, (128, 128))

  # Try to recognize the face
  prediction = model.predict(face_resize)
  print("prediction length", prediction[0], prediction[1])
  cv2.rectangle(new_img, (x, y), (x + w, y + h), (0, 255, 0), 3)
  # [1]
  # Write the name of recognized face
  cv2.putText(new_img,str(names[prediction[0]]),(x-20, y-10), cv2.FONT_ITALIC,1,(102, 255, 0))
  cv2.imwrite(save_path+'predict_{}.jpg'.format(str(names[prediction[0]])), new_img)
  return prediction[0],prediction[1]

who,_=detect_face(model2,haar_cascade,new_image,1, '/dbfs/mnt/root/')
print("predicted to be {}".format(names[int(who)]))

('grayim shape', (800, 591))
('face detector arr shape', (2, 4))
face at x:82, y:136,w:380,h:380
('prediction length', 0, 7105.931640710973)
predicted to be Angelina

In [14]:
#get images
male_dir="wasbs://mntdatabrick@zbatchaistorage.blob.core.windows.net/faces/male/"

female_dir="wasbs://mntdatabrick@zbatchaistorage.blob.core.windows.net/faces/female/"

images_male = spark.readImages(male_dir, recursive = True, sampleRatio = 0.1).cache()
images_female=spark.readImages(female_dir, recursive = True, sampleRatio = 0.1).cache()

In [16]:
images_male.printSchema()

root
-- image: struct (nullable = true)
 |-- path: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- width: integer (nullable = true)
 |-- type: integer (nullable = true)
 |-- bytes: binary (nullable = true)

In [17]:
maleStream = spark.streamImages(male_dir + "/*/*", sampleRatio = 0.9 )
male_images_query=maleStream.select(["image.path","image.height","image.width"]).writeStream.format("memory").queryName("male").start()
femaleStream = spark.streamImages(female_dir + "/*/*", sampleRatio = 0.9 )
female_images_query =femaleStream.select(["image.path","image.height","image.width"]).writeStream.format("memory").queryName("female").start()


In [18]:
male_images_query.stop()
female_images_query.stop()

In [19]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
male_data = images_male.take(3)
print(type(male_data))# take first three rows of the dataframe
im = male_data[2][0]
print(type(im)) # the image is in the first column of a given row

print("image type: {}, number of fields: {}".format(type(im), len(im)))
print("image path: {}".format(im.path))
print("height: {}, width: {}, OpenCV type: {}".format(im.height, im.width, im.type))
display(plt.imshow(arr))
arr = toNDArray(im)  
Image.fromarray(arr, "RGB")   # display the image inside notebook
im=Image.fromarray(toNDArray(im), "RGB")


--------------------------------------------------------------------------- 
 Exception Traceback (most recent call last)
 <command-1277953500304288> in <module> () 
 10 print ( "image path: {}" . format ( im . path ) ) 
 11 print ( "height: {}, width: {}, OpenCV type: {}" . format ( im . height , im . width , im . type ) ) 
 ---> 12 display ( plt . imshow ( arr ) ) 
 13 arr = toNDArray ( im ) 
 14 Image . fromarray ( arr , "RGB" ) # display the image inside notebook 

 /tmp/1534249600390-0/PythonShell.py in display (self, input, *args, **kwargs) 
 829 input . help ( ) # This is going to display the help as a side-effect 
 830 else : 
 --> 831 raise Exception ( genericErrorMsg ) 
 832 
 833 def displayHTML ( self , html ) : 

 Exception : Cannot call display(<class 'matplotlib.image.AxesImage'>)
 Call help(display) for more info.

In [20]:
def plot_img(figtitle,subtitle,img1,site):
  fig = plt.figure(figtitle, figsize=(10, 5))
  plt.suptitle(subtitle,fontsize=24)
  ax = fig.add_subplot(1, 2, 1)
  ax.set_title("Base",fontsize=12)
  plt.imshow(img1)
  plt.axis("off")
  ax = fig.add_subplot(1, 2, 2)
  ax.set_title(site,fontsize=12)
  plt.imshow(img2)
  plt.axis("off")

  display(plt.show())

In [21]:
from mmlspark import ImageTransformer

tr = (ImageTransformer()                  # images are resized and then cropped
      .setOutputCol("transformed")
      .resize(height = 200, width = 200)
      .crop(0, 0, height = 180, width = 180) )

small_male = tr.transform(images_male).select("transformed")

im = small_male.take(3)[2][0]                  # take third image
Image.fromarray(toNDArray(im), "RGB")

Out[ 66 ]: <PIL.Image.Image image mode=RGB size=180x180 at 0x7F04CDA71438>